In [78]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
#nltk.download('stopwords')
#nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/choldawa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/choldawa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [79]:
# import pip
# pip.main(['install', 'gensim==3.4.0'])

In [8]:
doc1 = "Sugar is bad to consume. My sister likes to have sugar, but not my father."
doc2 = "My father spends a lot of time driving my sister around to dance practice."
doc3 = "Doctors suggest that driving may cause increased stress and blood pressure."
doc4 = "Sometimes I feel pressure to perform well at school, but my father never seems to drive my sister to do better."
doc5 = "Health experts say that Sugar is not good for your lifestyle."

# compile documents
doc_complete = [doc1, doc2, doc3, doc4, doc5]

In [9]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop]) #take out stop words
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude) #remove punctuation
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split()) #lemmatize all words
    return normalized

doc_clean = [clean(doc).split() for doc in doc_complete]

In [38]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/choldawa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [39]:
#lemmatize: find roots and stems
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
#remove stopwords and words less than 3 letters
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [45]:
#get tect from top 10 docs
documents['headline_text'][:10]

0    aba decides against community broadcasting lic...
1       act fire witnesses must be aware of defamation
2       a g calls for infrastructure protection summit
3             air nz staff in aust strike for pay rise
4        air nz strike to affect australian travellers
5                    ambitious olsson wins triple jump
6           antic delighted with record breaking barca
7    aussie qualifier stosur wastes four memphis match
8         aust addresses un security council over iraq
9           australia is locked into war timetable opp
Name: headline_text, dtype: object

In [80]:
docs = documents['headline_text'][:100000].map(preprocess)
docs

0         [decid, communiti, broadcast, licenc]
1                            [wit, awar, defam]
2        [call, infrastructur, protect, summit]
3                   [staff, aust, strike, rise]
4          [strike, affect, australian, travel]
                          ...                  
99995       [rethink, foreign, tourism, invest]
99996                    [timber, secur, boost]
99997                       [power, like, fail]
99998    [cambodian, parti, seal, coalit, deal]
99999         [capriati, confid, serena, trick]
Name: headline_text, Length: 100000, dtype: object

In [81]:
#testing the preprocessing 
doc_sample = documents[documents['index'] == 10].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['australia', 'to', 'contribute', '10', 'million', 'in', 'aid', 'to', 'iraq']


 tokenized and lemmatized document: 
['australia', 'contribut', 'million', 'iraq']


In [82]:
dictionary = gensim.corpora.Dictionary(docs)
# count = 0
# for k, v in dictionary.iteritems():
#     print(k, v)
#     count += 1
#     if count > 10:
#         break

adding document #0 to Dictionary(0 unique tokens: [])
adding document #10000 to Dictionary(6518 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #20000 to Dictionary(8959 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #30000 to Dictionary(10693 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #40000 to Dictionary(12041 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #50000 to Dictionary(13150 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #60000 to Dictionary(14056 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #70000 to Dictionary(14863 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #80000 to Dictionary(15575 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)
adding document #900

In [83]:
#filter tokens that appear in less than 15 docs, and not in more than 50%, keep top 100k
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

discarding 13146 tokens: [('ambiti', 6), ('olsson', 2), ('antic', 7), ('memphi', 6), ('robson', 14), ('bathhous', 3), ('paroo', 3), ('blizzard', 4), ('brigadi', 3), ('overtim', 13)]...
keeping 3771 tokens which were in no less than 15 and no more than 50000 (=50.0%) documents
resulting dictionary: Dictionary(3771 unique tokens: ['broadcast', 'communiti', 'decid', 'licenc', 'awar']...)


In [84]:
bow_corpus = [dictionary.doc2bow(doc) for doc in docs]

bow_doc_10 = bow_corpus[10]
for i in range(len(bow_doc_10)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_10[i][0], 
                                               dictionary[bow_doc_10[i][0]], 
bow_doc_10[i][1]))

Word 32 ("iraq") appears 1 time.
Word 34 ("australia") appears 1 time.
Word 37 ("contribut") appears 1 time.
Word 38 ("million") appears 1 time.


In [85]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=7, id2word=dictionary, passes=2, workers=2)

using symmetric alpha at 0.14285714285714285
using symmetric eta at 0.14285714285714285
using serial LDA version on this node
running online LDA training, 7 topics, 2 passes over the supplied corpus of 100000 documents, updating every 4000 documents, evaluating every ~40000 documents, iterating 50x with a convergence threshold of 0.001000
training LDA model using 2 processes
PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/100000, outstanding queue size 1
PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/100000, outstanding queue size 2
PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/100000, outstanding queue size 3
PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/100000, outstanding queue size 4
PROGRESS: pass 0, dispatched chunk #4 = documents up to #10000/100000, outstanding queue size 5
PROGRESS: pass 0, dispatched chunk #5 = documents up to #12000/100000, outstanding queue size 6
PROGRESS: pass 0, dispatched chunk #6 = documents 

topic #5 (0.143): 0.010*"urg" + 0.010*"open" + 0.010*"tour" + 0.008*"protest" + 0.008*"claim" + 0.008*"charg" + 0.007*"govt" + 0.006*"lead" + 0.006*"fall" + 0.006*"school"
topic #1 (0.143): 0.015*"polic" + 0.014*"concern" + 0.008*"trial" + 0.007*"sydney" + 0.006*"health" + 0.006*"court" + 0.006*"charg" + 0.006*"consid" + 0.006*"fight" + 0.005*"get"
topic #2 (0.143): 0.021*"iraq" + 0.014*"govt" + 0.013*"kill" + 0.011*"council" + 0.010*"warn" + 0.008*"fund" + 0.008*"budget" + 0.008*"hospit" + 0.007*"consid" + 0.007*"plan"
topic #0 (0.143): 0.026*"polic" + 0.015*"govt" + 0.010*"urg" + 0.008*"probe" + 0.007*"investig" + 0.007*"begin" + 0.007*"rule" + 0.006*"farmer" + 0.006*"road" + 0.006*"council"
topic diff=0.222311, rho=0.277350
PROGRESS: pass 0, dispatched chunk #19 = documents up to #40000/100000, outstanding queue size 6
PROGRESS: pass 0, dispatched chunk #20 = documents up to #42000/100000, outstanding queue size 6
merging changes from 4000 documents into a model of 100000 documents


topic #1 (0.143): 0.012*"sydney" + 0.012*"concern" + 0.011*"polic" + 0.009*"trial" + 0.008*"worker" + 0.007*"strike" + 0.007*"minist" + 0.006*"union" + 0.006*"consid" + 0.006*"charg"
topic #2 (0.143): 0.021*"iraq" + 0.018*"kill" + 0.015*"council" + 0.014*"govt" + 0.012*"hospit" + 0.012*"warn" + 0.011*"plan" + 0.009*"consid" + 0.009*"fund" + 0.008*"attack"
topic #3 (0.143): 0.016*"polic" + 0.015*"report" + 0.014*"plan" + 0.011*"claim" + 0.011*"death" + 0.010*"govt" + 0.009*"miss" + 0.007*"search" + 0.007*"inquiri" + 0.007*"kill"
topic #5 (0.143): 0.012*"open" + 0.009*"urg" + 0.009*"tour" + 0.008*"protest" + 0.008*"melbourn" + 0.007*"fall" + 0.007*"presid" + 0.007*"claim" + 0.007*"govt" + 0.007*"start"
topic diff=0.120006, rho=0.192450
PROGRESS: pass 0, dispatched chunk #33 = documents up to #68000/100000, outstanding queue size 6
PROGRESS: pass 0, dispatched chunk #34 = documents up to #70000/100000, outstanding queue size 6
merging changes from 4000 documents into a model of 100000 doc

topic #0 (0.143): 0.033*"polic" + 0.017*"govt" + 0.014*"probe" + 0.012*"urg" + 0.010*"investig" + 0.008*"latham" + 0.007*"rule" + 0.007*"begin" + 0.007*"road" + 0.007*"farmer"
topic #3 (0.143): 0.020*"polic" + 0.019*"report" + 0.013*"miss" + 0.013*"claim" + 0.012*"death" + 0.011*"plan" + 0.009*"govt" + 0.009*"search" + 0.007*"inquiri" + 0.007*"south"
topic #1 (0.143): 0.017*"sydney" + 0.011*"concern" + 0.010*"polic" + 0.009*"trial" + 0.009*"worker" + 0.009*"union" + 0.008*"bird" + 0.008*"minist" + 0.008*"strike" + 0.007*"plan"
topic #6 (0.143): 0.020*"court" + 0.019*"face" + 0.016*"charg" + 0.012*"jail" + 0.008*"clash" + 0.007*"decis" + 0.007*"assault" + 0.006*"chief" + 0.006*"year" + 0.006*"prison"
topic diff=0.098681, rho=0.156174
PROGRESS: pass 0, dispatched chunk #47 = documents up to #96000/100000, outstanding queue size 6
PROGRESS: pass 0, dispatched chunk #48 = documents up to #98000/100000, outstanding queue size 6
merging changes from 4000 documents into a model of 100000 docu

topic #4 (0.143): 0.011*"world" + 0.010*"back" + 0.010*"lead" + 0.010*"home" + 0.010*"test" + 0.009*"australia" + 0.008*"final" + 0.007*"plan" + 0.007*"council" + 0.006*"water"
topic #2 (0.143): 0.041*"iraq" + 0.018*"kill" + 0.016*"council" + 0.014*"hospit" + 0.013*"warn" + 0.012*"plan" + 0.012*"govt" + 0.010*"fund" + 0.009*"attack" + 0.008*"troop"
topic #0 (0.143): 0.033*"polic" + 0.018*"govt" + 0.014*"urg" + 0.012*"probe" + 0.011*"investig" + 0.008*"begin" + 0.008*"rule" + 0.007*"road" + 0.007*"farmer" + 0.006*"welcom"
topic #6 (0.143): 0.020*"face" + 0.020*"court" + 0.018*"charg" + 0.012*"jail" + 0.009*"clash" + 0.009*"iraqi" + 0.009*"prison" + 0.008*"appeal" + 0.007*"decis" + 0.006*"drug"
topic diff=0.090517, rho=0.138675
PROGRESS: pass 1, dispatched chunk #11 = documents up to #24000/100000, outstanding queue size 6
PROGRESS: pass 1, dispatched chunk #12 = documents up to #26000/100000, outstanding queue size 6
merging changes from 4000 documents into a model of 100000 documents
t

topic #0 (0.143): 0.033*"polic" + 0.020*"govt" + 0.015*"urg" + 0.012*"probe" + 0.011*"investig" + 0.009*"rule" + 0.009*"begin" + 0.008*"farmer" + 0.007*"road" + 0.007*"solomon"
topic #2 (0.143): 0.028*"iraq" + 0.020*"council" + 0.018*"kill" + 0.015*"plan" + 0.014*"warn" + 0.014*"govt" + 0.012*"hospit" + 0.012*"fund" + 0.011*"consid" + 0.009*"industri"
topic #3 (0.143): 0.023*"report" + 0.018*"polic" + 0.016*"claim" + 0.014*"death" + 0.014*"miss" + 0.009*"inquiri" + 0.009*"search" + 0.008*"korea" + 0.008*"plan" + 0.008*"bomb"
topic diff=0.082407, rho=0.138675
PROGRESS: pass 1, dispatched chunk #24 = documents up to #50000/100000, outstanding queue size 5
PROGRESS: pass 1, dispatched chunk #25 = documents up to #52000/100000, outstanding queue size 6
PROGRESS: pass 1, dispatched chunk #26 = documents up to #54000/100000, outstanding queue size 6
merging changes from 4000 documents into a model of 100000 documents
topic #6 (0.143): 0.026*"face" + 0.023*"court" + 0.020*"charg" + 0.012*"jai

topic #3 (0.143): 0.023*"report" + 0.021*"polic" + 0.016*"claim" + 0.016*"miss" + 0.014*"death" + 0.010*"search" + 0.009*"kill" + 0.009*"crash" + 0.008*"bomb" + 0.008*"arrest"
topic #0 (0.143): 0.037*"polic" + 0.021*"govt" + 0.016*"urg" + 0.014*"probe" + 0.011*"investig" + 0.009*"road" + 0.009*"begin" + 0.009*"rule" + 0.008*"farmer" + 0.007*"seek"
topic #1 (0.143): 0.017*"sydney" + 0.014*"concern" + 0.011*"trial" + 0.011*"worker" + 0.010*"strike" + 0.010*"union" + 0.009*"plan" + 0.008*"boost" + 0.007*"region" + 0.007*"fear"
topic #2 (0.143): 0.025*"iraq" + 0.021*"council" + 0.020*"kill" + 0.017*"plan" + 0.015*"hospit" + 0.015*"warn" + 0.013*"govt" + 0.011*"fund" + 0.011*"consid" + 0.010*"attack"
topic diff=0.078883, rho=0.138675
PROGRESS: pass 1, dispatched chunk #39 = documents up to #80000/100000, outstanding queue size 6
PROGRESS: pass 1, dispatched chunk #40 = documents up to #82000/100000, outstanding queue size 6
merging changes from 4000 documents into a model of 100000 document

topic #0 (0.143): 0.035*"polic" + 0.021*"govt" + 0.016*"urg" + 0.013*"probe" + 0.012*"investig" + 0.009*"road" + 0.009*"begin" + 0.008*"rule" + 0.008*"farmer" + 0.008*"seek"
topic #6 (0.143): 0.027*"charg" + 0.026*"face" + 0.026*"court" + 0.013*"jail" + 0.011*"prison" + 0.010*"appeal" + 0.009*"murder" + 0.008*"clash" + 0.008*"year" + 0.008*"decis"
topic diff=0.067302, rho=0.138675
merging changes from 4000 documents into a model of 100000 documents
topic #0 (0.143): 0.036*"polic" + 0.022*"govt" + 0.017*"urg" + 0.013*"probe" + 0.012*"investig" + 0.009*"road" + 0.009*"begin" + 0.009*"rule" + 0.008*"seek" + 0.008*"drug"
topic #2 (0.143): 0.030*"iraq" + 0.023*"council" + 0.018*"kill" + 0.018*"plan" + 0.016*"warn" + 0.016*"hospit" + 0.013*"fund" + 0.013*"govt" + 0.011*"industri" + 0.011*"attack"
topic #6 (0.143): 0.027*"charg" + 0.027*"court" + 0.026*"face" + 0.013*"jail" + 0.012*"prison" + 0.011*"appeal" + 0.009*"murder" + 0.008*"clash" + 0.008*"decis" + 0.008*"year"
topic #4 (0.143): 0.01

In [86]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

topic #0 (0.143): 0.036*"polic" + 0.022*"govt" + 0.017*"urg" + 0.013*"probe" + 0.012*"investig" + 0.009*"road" + 0.009*"begin" + 0.009*"rule" + 0.008*"seek" + 0.008*"drug"
topic #1 (0.143): 0.015*"concern" + 0.014*"sydney" + 0.011*"worker" + 0.011*"strike" + 0.011*"union" + 0.010*"trial" + 0.010*"plan" + 0.009*"boost" + 0.009*"terror" + 0.007*"fear"
topic #2 (0.143): 0.030*"iraq" + 0.023*"council" + 0.018*"kill" + 0.018*"plan" + 0.016*"warn" + 0.016*"hospit" + 0.013*"fund" + 0.013*"govt" + 0.011*"industri" + 0.011*"attack"
topic #3 (0.143): 0.025*"report" + 0.021*"polic" + 0.017*"claim" + 0.015*"miss" + 0.012*"death" + 0.010*"kill" + 0.009*"search" + 0.009*"bomb" + 0.008*"suspect" + 0.008*"south"
topic #4 (0.143): 0.014*"test" + 0.014*"australia" + 0.012*"lead" + 0.012*"home" + 0.012*"back" + 0.010*"world" + 0.009*"final" + 0.008*"play" + 0.007*"compani" + 0.007*"care"
topic #5 (0.143): 0.019*"open" + 0.014*"elect" + 0.012*"protest" + 0.011*"tour" + 0.011*"break" + 0.010*"die" + 0.009*

In [87]:
#get topic composition of new doc
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.5693705677986145	 Topic: 0.030*"iraq" + 0.023*"council" + 0.018*"kill" + 0.018*"plan" + 0.016*"warn"
Score: 0.2873418629169464	 Topic: 0.027*"charg" + 0.027*"court" + 0.026*"face" + 0.013*"jail" + 0.012*"prison"
Score: 0.02871878445148468	 Topic: 0.015*"concern" + 0.014*"sydney" + 0.011*"worker" + 0.011*"strike" + 0.011*"union"
Score: 0.028713848441839218	 Topic: 0.036*"polic" + 0.022*"govt" + 0.017*"urg" + 0.013*"probe" + 0.012*"investig"
Score: 0.02867109142243862	 Topic: 0.025*"report" + 0.021*"polic" + 0.017*"claim" + 0.015*"miss" + 0.012*"death"
Score: 0.028610959649086	 Topic: 0.019*"open" + 0.014*"elect" + 0.012*"protest" + 0.011*"tour" + 0.011*"break"
Score: 0.02857288159430027	 Topic: 0.014*"test" + 0.014*"australia" + 0.012*"lead" + 0.012*"home" + 0.012*"back"
